## **Final Assignment 2 - Create Dashboard using Plotly and Dash**

### **Scenario:**

The objective of this part of the Final Assignment is to analyze the historical trends in automobile sales during recession periods, as you did in the previous part. The goal is to provide insights into how the sales of XYZAutomotives, a company specializing in automotive sales, were affected during times of recession.

In this final assignment, you will have the opportunity to demonstrate the **Dashboarding** skills you have acquired in this course. This lab aims to assess your abilities in creating various visualizations using Plotly and Dash. 

As a data scientist, you have been given a task to prepare a report on your finding from Automobile Sales data analysis. You decided to develop a dashboard representing two main reports:-

- Yearly Automobile Sales Statistics
- Recession Period Statistics

NOTE: Year range is between 1980 and 2013.


In [ ]:
# import required libraries

import pandas as pd
import plotly.express as px
from dash import dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [ ]:
# Load dataset

path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/historical_automobile_sales.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
df.info()

In [ ]:
# Create years list
years = [i for i in range(1980, 2024, 1)]

In [ ]:
# Create dropdown widgets
report_type = [{'label': 'Yearly Statistics', 'value':'Yearly Statistics'},
               {'label': 'Recession Period Statistics', 'value':'Recession Statistics'}]

year_dropdown = [{'label': i, 'value': i} for i in years]

In [ ]:
# Create dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [ ]:
# Create app layout

app.layout = html.Div([
    # Title of dashboard
    html.H1('Automobile Sales Statistics Dashboard',
            style={'textAlign':'center', 'color':'#503D36', 'font-size':24}),
    # Div for 1st dropdown
    html.Div([
        html.Label('Select Statistics'),
        dcc.Dropdown(id='dropdown-statistics',
                     options=report_type,
                     placeholder='Select a report type')
    ]),#,style={'width':'80%', 'padding':'3px', 'font-size':20, 'text-align-last':'center'}),
    #html.Br(),
    # Div for second dropdown
    html.Div([
        html.Label('Select Year'),
        dcc.Dropdown(id='select-year',
                     options=year_dropdown,
                     value=df.Year.min())
    ]),#,style={'width':'80%', 'padding':'3px', 'font-size':20, 'text-align-last':'center'}),
    #html.Br(),
    # Div for Graphs
    html.Div([
        html.Div(
            id='output-container', className='chart-grid', style={'display':'flex'}
        )
    ]),

])

In [ ]:
# Add callbacks and callback functions

# 1st callback for selecting report type, we will use a chained callback where the output of one component serves as the input for another
@app.callback(Output(component_id='select-year',component_property='disabled'),
              Input(component_id='dropdown-statistics', component_property='value'))
def get_statistics(selected_statistics):
    if (selected_statistics == 'Yearly Statistics') or (selected_statistics == 'Recession Statistics'):
        return False
    else:
        return True
    


In [ ]:
# 2nd callback for displaying and updating graphs

@app.callback(Output(component_id='output-container', component_property='children'),
              [Input(component_id='select-year', component_property='value'),
               Input(component_id='dropdown-statistics', component_property='value')])
def update_output_container(input_year, selected_statistics):
    if (selected_statistics == 'Recession Statistics'):
        # Filter the data for recession periods
        recession_data = df.query('Recession == 1 and Year == @input_year')

        # Create plots for Recession Statistics Report
        # Plot 1 - Average automobile Sales using line chart
        rec_sales = df.query('Recession == 1').groupby('Year')['Automobile_Sales'].mean().reset_index()
        Rchart1 = dcc.Graph(
            figure=px.line(rec_sales,
                           x='Year',
                           y='Automobile_Sales',
                           title='Average Automobile Sales per Year')
                           )

        # Plot 2 - Average no. of vehicles sold by vehicle type using bar chart
        vehicle_sales = recession_data.groupby(['Year', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        Rchart2 = dcc.Graph(
            figure=px.bar(vehicle_sales,
                           x='Vehicle_Type',
                           y='Automobile_Sales',
                           title=f'Average Sales per Vehicle for {input_year}')
                           )

        # Plot 3 - Total expenditure share by vehicle type during recession using pie chart
        vehicle_exp = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Rchart3 = dcc.Graph(
            figure=px.pie(vehicle_exp,
                           names='Vehicle_Type',
                           values='Advertising_Expenditure',
                           title=f'Total Advertising Expenditure per vehicle for {input_year}')
                           )

        # Plot 4 - Effect of unemployment rate on vehicle type and sales using bar chart
        unemployment_effect = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].sum().reset_index() 
        Rchart4 = dcc.Graph(
            figure=px.bar(unemployment_effect,
                           x='unemployment_rate',
                           y='Automobile_Sales',
                           color='Vehicle_Type',
                           title=f'Effect of Unemployment Rate on Vehicle Type for {input_year}')
                           )
        
        return [
            html.Div(className='chart-item',children=[html.Div(children=Rchart1),html.Div(children=Rchart2)]),
            html.Div(className='chart-item',children=[html.Div(children=Rchart3),html.Div(children=Rchart4)])
        ]

    # Yearly Report Statistics
    elif (input_year and selected_statistics == 'Yearly Statistics'):
        # Filter the data for years
        yearly_data = df.query('Year == @input_year')
        
        # Create plots for Yearly Report Statistics
        # Plot 1 - Yearly Automobile Sales using line chart
        yearly_sales = df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Ychart1 = dcc.Graph(
            figure=px.line(yearly_sales, x='Year', y='Automobile_Sales',title='Yearly Automobile Sales')
        )
        
        # Plot 2 - Total Monthly Automobile Sales using line chart
        monthly_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Ychart2 = dcc.Graph(
            figure=px.line(monthly_sales,x='Month', y='Automobile_Sales', title=f'Monthly Automobile Sales for {input_year}')
        )

        # Plot 3 - Average Monthly Automobile Sales of each vehicle type using bar chart
        m_sales_per_vehicle = yearly_data.groupby(['Month', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        Ychart3 = dcc.Graph(
            figure=px.bar(m_sales_per_vehicle,x='Month', y='Automobile_Sales',color='Vehicle_Type', title=f'Monthly Automobile Sales per Vehicle Type for {input_year}')
        )

        # Plot 4 - Total Advertisement Expenditure for each Vehicle using horizontal bar chart
        advert_exp = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index().sort_values(by='Advertising_Expenditure',ascending=True)
        Ychart4 = dcc.Graph(
            figure=px.bar(advert_exp,x='Vehicle_Type', y='Advertising_Expenditure', orientation='h', title=f'Total Advertising Expenditure per Vehicle for {input_year}')
        )

        return [
            html.Div(className='chart-item',children=[html.Div(children=Ychart1),html.Div(children=Ychart2)]),
            html.Div(className='chart-item',children=[html.Div(children=Ychart3),html.Div(children=Ychart4)])
        ]
    
    else:
        return html.Div()
        

In [ ]:
if __name__ == '__main__':
    app.run_server(jupyter_mode = 'tab',port=4040)